In [ ]:
import tensorflow as tf
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import random

In [ ]:
imagePaths = list(paths.list_images("../small-casia/"))
names = []
for imagePath in imagePaths:
    names.append(imagePath.split("/")[-2])
print(set(names))
names = set(names)

In [ ]:
print(len(imagePaths))
print(len(names))

In [ ]:
os.mkdir("../sm_train_1")
os.mkdir("../sm_test_1")

In [ ]:
for name in names:
    path = "../small-casia/"+name
    write_path = "../sm_train_1/"+str(name)
    write_path2 = "../sm_test_1/"+str(name)
    os.mkdir(write_path)
    os.mkdir(write_path2)
    pathimages = list(paths.list_images(path))
    size = len(pathimages)
    train_size = int(0.7 * size)
    test_size = size - train_size
    print("Train Size for",name,train_size)
    print("Test Size for",name, test_size)
    a = random.sample(range(0,size),size)
    for i in range(0,size):
        if i < train_size:
            path_name = write_path+"/"+str(i)+".jpg"
            img = cv2.imread(pathimages[a[i]])
            print(pathimages[a[i]],"at",path_name)
            cv2.imwrite(path_name,img)
        else:
            path_name = write_path2+"/"+str(i)+".jpg"
            img = cv2.imread(pathimages[a[i]])
            print(pathimages[a[i]],"at",path_name)
            cv2.imwrite(path_name,img)

In [ ]:
imagePaths = list(paths.list_images("../sm_train_1/"))

In [ ]:
len(imagePaths)

In [ ]:
from src import Detection
from src import Recognition
from src import preprocess

In [ ]:
rec = Recognition.Recognition("/home/santoshv/RT_facenet/Models/Recognition_facenet.pb")
dec = Detection.Detection("/home/santoshv/RT_facenet/Models/Detection_mtcnn.pb")

In [ ]:
nam = []
emb = []
for (i,imagePath) in enumerate(imagePaths):
    name = imagePath.split("/")[-2]
    image = cv2.imread(imagePath)
    bbox,conf,landmarks = dec.detect(image)
    for box in bbox:
        box = box.astype('int32')
        face = image[box[0]:box[2] , box[1]:box[3]] 
        if face.shape[0] !=0 and face.shape[1] !=0 and face.shape[2] !=0:
            face = preprocess.prewhiten(face)
            embedding = rec.recognize(face)
            nam.append(name)
            emb.append(embedding)
            print(i)

In [ ]:
import pickle
from sklearn.svm import SVC
emb_arrays = np.array(emb).reshape(np.array(emb).shape[0], np.array(emb).shape[2])
labels = np.array(nam)
classnames = list(set(labels))
model = SVC(kernel='linear', probability=True)
model.fit(emb_arrays, labels)
print(model,classnames)
with open("/home/santoshv/RT_facenet/util/classifier_3.pkl", 'wb') as outfile:
    pickle.dump((model,classnames), outfile)

In [ ]:
imagePaths = list(paths.list_images("../sm_test_1/"))

In [ ]:
len(imagePaths)

In [ ]:
import pickle
with open("/home/santoshv/RT_facenet/util/classifier_3.pkl", 'rb') as infile:
    model,class_names = pickle.load(infile)

In [ ]:
model

In [ ]:
correct = 0
wrong = 0
for (i,imagePath) in enumerate(imagePaths):
    img = cv2.imread(imagePath)
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    imgp = preprocess.prewhiten(img)
    embd = rec.recognize(imgp)
    original_name = imagePath.split("/")[-2]
    print("Original Name:",original_name)
    predict_name = model.predict(embd)
    print("Predicted Name:",predict_name[0])
    if (predict_name == original_name):
        correct = correct+1
        print("correct", correct)
    else:
        wrong = wrong+1
        plt.imshow(img)
        plt.show()
        print("Wrong",wrong)